In [4]:
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

def fetch_data(ticker="MXN=X", lookback_days=365 * 3):
    """Fetch historical exchange rate data."""
    from datetime import datetime, timedelta
    end_date = datetime.now()
    start_date = end_date - timedelta(days=lookback_days)
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date.strftime("%Y-%m-%d"), 
                       end=end_date.strftime("%Y-%m-%d"), 
                       interval='1d')
    return df[['Close']].dropna()

def perform_eda(df):
    st.title("Exploratory Data Analysis (EDA) - MXN/USD Exchange Rate")
    
    st.subheader("Data Overview")
    st.write(df.describe())
    
    st.subheader("Missing Values Check")
    st.write(df.isnull().sum())
    
    st.subheader("Closing Price Distribution")
    fig, ax = plt.subplots()
    sns.histplot(df['Close'], bins=30, kde=True, ax=ax)
    ax.set_title("Distribution of Closing Prices")
    st.pyplot(fig)
    
    st.subheader("Rolling Mean and Volatility")
    df['Rolling_Mean'] = df['Close'].rolling(window=30).mean()
    df['Volatility'] = df['Close'].rolling(window=30).std()
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Close'], label='Closing Price', alpha=0.5)
    ax.plot(df.index, df['Rolling_Mean'], label='30-day Rolling Mean', color='red')
    ax.set_title("Closing Price with 30-Day Moving Average")
    ax.legend()
    st.pyplot(fig)
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Volatility'], label='30-day Volatility', color='green')
    ax.set_title("30-Day Rolling Volatility")
    ax.legend()
    st.pyplot(fig)
    
    st.subheader("Correlation Matrix")
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear
    corr_matrix = df.corr()
    fig, ax = plt.subplots()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', ax=ax)
    ax.set_title("Correlation Matrix")
    st.pyplot(fig)

def compare_models(df):
    st.subheader("Model Comparison: Linear Regression vs Decision Tree")
    df['S_3'] = df['Close'].rolling(window=3).mean()
    df['S_9'] = df['Close'].rolling(window=9).mean()
    df['volatility'] = df['Close'].rolling(window=5).std()
    df['next_day_rate'] = df['Close'].shift(-1)
    df = df.dropna()
    
    X = df[['S_3', 'S_9', 'volatility', 'day_of_week', 'month', 'day_of_year']]
    y = df['next_day_rate']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Linear Regression Model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_preds = lr_model.predict(X_test)
    
    # Decision Tree Model
    dt_model = DecisionTreeRegressor()
    dt_model.fit(X_train, y_train)
    dt_preds = dt_model.predict(X_test)
    
    # Metrics
    metrics = {
        "Linear Regression": {
            "MSE": mean_squared_error(y_test, lr_preds),
            "MAE": mean_absolute_error(y_test, lr_preds),
            "R²": lr_model.score(X_test, y_test)
        },
        "Decision Tree": {
            "MSE": mean_squared_error(y_test, dt_preds),
            "MAE": mean_absolute_error(y_test, dt_preds),
            "R²": dt_model.score(X_test, y_test)
        }
    }
    
    st.write(pd.DataFrame(metrics))
    
    fig, ax = plt.subplots()
    ax.plot(y_test.index, y_test, label='Actual Rate', color='blue')
    ax.plot(y_test.index, lr_preds, label='Linear Regression Predictions', color='red', linestyle='dashed')
    ax.plot(y_test.index, dt_preds, label='Decision Tree Predictions', color='green', linestyle='dotted')
    ax.set_title("Model Predictions vs Actual Rates")
    ax.legend()
    st.pyplot(fig)

def main():
    st.set_page_config(page_title="EDA - MXN/USD Exchange Rate", layout="wide")
    df = fetch_data()
    perform_eda(df)
    compare_models(df)

if __name__ == "__main__":
    main()


2025-03-01 21:51:18.248 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:51:18.676 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:51:18.825 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-01 21:51:18.825 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:51:18.826 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:51:18.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:51:18.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored whe

new EDA

In [5]:
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

def fetch_data(ticker="MXN=X", lookback_days=365 * 3):
    """Fetch historical exchange rate data."""
    from datetime import datetime, timedelta
    end_date = datetime.now()
    start_date = end_date - timedelta(days=lookback_days)
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date.strftime("%Y-%m-%d"), 
                       end=end_date.strftime("%Y-%m-%d"), 
                       interval='1d')
    return df[['Close']].dropna()

def perform_eda(df):
    st.title("Exploratory Data Analysis (EDA) - MXN/USD Exchange Rate")
    
    st.subheader("Data Overview")
    st.write(df.describe())
    
    st.subheader("Missing Values Check")
    st.write(df.isnull().sum())
    
    st.subheader("Closing Price Distribution")
    fig, ax = plt.subplots()
    sns.histplot(df['Close'], bins=30, kde=True, ax=ax)
    ax.set_title("Distribution of Closing Prices")
    st.pyplot(fig)
    
    st.subheader("Rolling Mean and Volatility")
    df['Rolling_Mean'] = df['Close'].rolling(window=30).mean()
    df['Volatility'] = df['Close'].rolling(window=30).std()
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Close'], label='Closing Price', alpha=0.5)
    ax.plot(df.index, df['Rolling_Mean'], label='30-day Rolling Mean', color='red')
    ax.set_title("Closing Price with 30-Day Moving Average")
    ax.legend()
    st.pyplot(fig)
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Volatility'], label='30-day Volatility', color='green')
    ax.set_title("30-Day Rolling Volatility")
    ax.legend()
    st.pyplot(fig)
    
    st.subheader("Correlation Matrix")
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear
    corr_matrix = df.corr()
    fig, ax = plt.subplots()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', ax=ax)
    ax.set_title("Correlation Matrix")
    st.pyplot(fig)
    
    st.subheader("Trend Over Time")
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Close'], label='Closing Price', color='blue')
    ax.set_title("Exchange Rate Trend Over Time")
    ax.set_xlabel("Date")
    ax.set_ylabel("Exchange Rate")
    ax.legend()
    st.pyplot(fig)

def compare_models(df):
    st.subheader("Model Comparison: Linear Regression vs Decision Tree")
    df['S_3'] = df['Close'].rolling(window=3).mean()
    df['S_9'] = df['Close'].rolling(window=9).mean()
    df['volatility'] = df['Close'].rolling(window=5).std()
    df['next_day_rate'] = df['Close'].shift(-1)
    df = df.dropna()
    
    X = df[['S_3', 'S_9', 'volatility', 'day_of_week', 'month', 'day_of_year']]
    y = df['next_day_rate']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Linear Regression Model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_preds = lr_model.predict(X_test)
    
    # Decision Tree Model
    dt_model = DecisionTreeRegressor()
    dt_model.fit(X_train, y_train)
    dt_preds = dt_model.predict(X_test)
    
    # Metrics
    metrics = {
        "Linear Regression": {
            "MSE": mean_squared_error(y_test, lr_preds),
            "MAE": mean_absolute_error(y_test, lr_preds),
            "R²": lr_model.score(X_test, y_test)
        },
        "Decision Tree": {
            "MSE": mean_squared_error(y_test, dt_preds),
            "MAE": mean_absolute_error(y_test, dt_preds),
            "R²": dt_model.score(X_test, y_test)
        }
    }
    
    st.write(pd.DataFrame(metrics))
    
    fig, ax = plt.subplots()
    ax.plot(y_test.index, y_test, label='Actual Rate', color='blue')
    ax.plot(y_test.index, lr_preds, label='Linear Regression Predictions', color='red', linestyle='dashed')
    ax.plot(y_test.index, dt_preds, label='Decision Tree Predictions', color='green', linestyle='dotted')
    ax.set_title("Model Predictions vs Actual Rates")
    ax.legend()
    st.pyplot(fig)

def main():
    st.set_page_config(page_title="EDA - MXN/USD Exchange Rate", layout="wide")
    df = fetch_data()
    perform_eda(df)
    compare_models(df)

if __name__ == "__main__":
    main()


2025-03-01 21:53:09.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:09.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [6]:

    st.subheader("Trend Over Time")
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Close'], label='Closing Price', color='blue')
    ax.set_title("Exchange Rate Trend Over Time")
    ax.set_xlabel("Date")
    ax.set_ylabel("Exchange Rate")
    ax.legend()
    st.pyplot(fig)

2025-03-01 21:53:56.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:53:56.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


NameError: name 'df' is not defined

In [7]:
pip install yfinance pandas matplotlib seaborn numpy


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

def fetch_data(ticker="MXN=X", lookback_days=365 * 3):
    """Fetch historical exchange rate data."""
    from datetime import datetime, timedelta
    end_date = datetime.now()
    start_date = end_date - timedelta(days=lookback_days)
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date.strftime("%Y-%m-%d"), 
                       end=end_date.strftime("%Y-%m-%d"), 
                       interval='1d')
    return df[['Close']].dropna()

def perform_eda(df):
    st.title("Exploratory Data Analysis (EDA) - MXN/USD Exchange Rate")
    
    st.subheader("Data Overview")
    st.write(df.describe())
    
    st.subheader("Missing Values Check")
    st.write(df.isnull().sum())
    
    st.subheader("Closing Price Distribution")
    fig, ax = plt.subplots()
    sns.histplot(df['Close'], bins=30, kde=True, ax=ax)
    ax.set_title("Distribution of Closing Prices")
    st.pyplot(fig)
    
    st.subheader("Rolling Mean and Volatility")
    df['Rolling_Mean'] = df['Close'].rolling(window=30).mean()
    df['Volatility'] = df['Close'].rolling(window=30).std()
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Close'], label='Closing Price', alpha=0.5)
    ax.plot(df.index, df['Rolling_Mean'], label='30-day Rolling Mean', color='red')
    ax.set_title("Closing Price with 30-Day Moving Average")
    ax.legend()
    st.pyplot(fig)
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Volatility'], label='30-day Volatility', color='green')
    ax.set_title("30-Day Rolling Volatility")
    ax.legend()
    st.pyplot(fig)
    
    st.subheader("Correlation Matrix")
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear
    corr_matrix = df.corr()
    fig, ax = plt.subplots()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', ax=ax)
    ax.set_title("Correlation Matrix")
    st.pyplot(fig)
    
    st.subheader("Trend Over Time")
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df.index, df['Close'], label='Closing Price', color='blue')
    ax.set_title("Exchange Rate Trend Over Time")
    ax.set_xlabel("Date")
    ax.set_ylabel("Exchange Rate")
    ax.legend()
    st.pyplot(fig)
    
    st.subheader("Boxplot of Monthly Exchange Rate Variation")
    df['year_month'] = df.index.to_period('M')
    fig, ax = plt.subplots(figsize=(12, 5))
    sns.boxplot(x=df['year_month'].astype(str), y=df['Close'], ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_title("Monthly Exchange Rate Variation")
    st.pyplot(fig)

def main():
    st.set_page_config(page_title="EDA - MXN/USD Exchange Rate", layout="wide")
    df = fetch_data()
    perform_eda(df)

if __name__ == "__main__":
    main()


2025-03-01 21:57:36.565 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 21:57:36.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar